# Driving Sales with Data: A Chinook SQL Project

## Introduction 

This project will answer various business questions using SQL.

The database used is called Chinook, which is a sample database that represents a digital media store, including tables for artists, albums, media tracks, invoices, and customers.

* Media-related data was created using real data from an Apple iTunes library.
* Customer and employee information was created using fictitious names and addresses that can be located on Google maps, and other well formatted data (phone, fax, email, etc.)
* Sales information was auto generated using random data for a four year period.

The Chinook sample database includes:

* 11 tables
* A variety of indexes, primary and foreign key constraints
* Over 15,000 rows of data

A diagram of the databse schema can be found [here](https://s3.amazonaws.com/dq-content/191/chinook-schema.svg).

Connecting the Jupyter Notebook to the database file:

In [3]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

## Overview of the Data

Quering the database to get a list of all the tables and views in our database:

In [4]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table", "view")

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


## Selecting New Albums to Purchase

Hypothetical Scenario:

The Chinook record store has just signed a deal with a new record label, and I've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:

Artist Name and Genre
* Regal / Hip-Hop
* Red Tone / Punk
* Meteor and the Girls / Pop
* Slim Jim Bites / Blues

The record label specialises in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so I'm interested in finding out which genres sell the best in the USA.

In [5]:
%%sql

-- This common table expression (CTE) filters the invoice lines for tracks sold in the USA.
WITH USA_tracks_sold AS
    (SELECT il.*
     FROM invoice_line il
     INNER JOIN invoice i ON i.invoice_id = il.invoice_id
     INNER JOIN customer c ON c.customer_id = i.customer_id
     WHERE country == "USA"
    )

-- This main query calculates and presents the top-selling genres in the USA.
SELECT
    g.name genre,
    COUNT(uts.quantity) absolute_tracks_sold,
    CAST(COUNT(uts.quantity) AS FLOAT) / (SELECT COUNT(*) FROM USA_tracks_sold) percentage_tracks_sold
FROM USA_tracks_sold uts
INNER JOIN track t ON t.track_id = uts.track_id
INNER JOIN genre g ON g.genre_id = t.genre_id
GROUP BY g.name
ORDER BY absolute_tracks_sold DESC
LIMIT 10

Done.


genre,absolute_tracks_sold,percentage_tracks_sold
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Latin,22,0.02093244529019981
Pop,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


Based on the sales of tracks across different genres in the USA, we should purchase the new albums by the following artists:

* Red Tone (Punk)
* Slim Jim Bites (Blues)
* Meteor and Girls (Pop)

## Analysing Employee Sales Performance

Hypothetical Scenerio:

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. I have been asked to analyse the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

In [6]:
%%sql

-- This CTE calculates total sales per customer support representative.
WITH customer_support_sales AS
    (SELECT
         i.customer_id,
         c.support_rep_id,
         SUM(i.total) total
     FROM invoice i 
     INNER JOIN customer c ON i.customer_id = c.customer_id
     GROUP BY 1, 2
    )

-- This main query aggregates total sales per employee and presents the results.
SELECT 
    e.first_name || " " || e.last_name employee_name,
    e.hire_date,
    SUM(css.total) total_sales
FROM customer_support_sales css
INNER JOIN employee e ON e.employee_id = css.support_rep_id
GROUP BY employee_name

Done.


employee_name,hire_date,total_sales
Jane Peacock,2017-04-01 00:00:00,1731.5099999999998
Margaret Park,2017-05-03 00:00:00,1584.0000000000002
Steve Johnson,2017-10-17 00:00:00,1393.92


While there is a 20% difference in sales between Jane (the top employee) and Steve (the bottom employee), the difference roughly corresponds with the differences in their hiring dates.

## Analysing Sales by Country 

Hypothetical Scenerio:

My next task is to analyse the sales data for customers from each different country. I have been given guidance to use the country value from the customers table, and ignore the country from the billing address in the invoice table.

In particular, I have been directed to calculate data, for each country, on the:

* total number of customers
* total value of sales
* average value of sales per customer
* average order value

Because there are a number of countries with only one customer, you should group these customers as "Other" in your analysis. 

In [7]:
%%sql

-- This CTE categorises countries with only one customer as "Other".

WITH country_or_other AS
    (
     SELECT
       CASE
           WHEN (
                 SELECT COUNT(*)
                 FROM customer
                 WHERE country = c.country
                ) = 1 THEN "Other"
           ELSE c.country
       END AS country,
       c.customer_id,
       il.*
     FROM invoice_line il
     INNER JOIN invoice i ON i.invoice_id = il.invoice_id
     INNER JOIN customer c ON c.customer_id = i.customer_id
    )

-- This main query calculates various metrics grouped by country or "Other".
SELECT
    country,
    customers,
    total_sales,
    average_order,
    customer_lifetime_value
FROM
    (
    SELECT
        country,
        count(distinct customer_id) customers,
        SUM(unit_price) total_sales,
        SUM(unit_price) / count(distinct customer_id) customer_lifetime_value,
        SUM(unit_price) / count(distinct invoice_id) average_order,
        CASE
            WHEN country = "Other" THEN 1
            ELSE 0
        END AS sort
    FROM country_or_other
    GROUP BY country
    ORDER BY sort ASC, total_sales DESC
    );

Done.


country,customers,total_sales,average_order,customer_lifetime_value
USA,13,1040.490000000008,7.942671755725252,80.03769230769292
Canada,8,535.5900000000034,7.047236842105309,66.94875000000043
Brazil,5,427.68000000000245,7.011147540983647,85.53600000000048
France,5,389.0700000000021,7.781400000000042,77.81400000000042
Germany,4,334.6200000000016,8.161463414634186,83.6550000000004
Czech Republic,2,273.24000000000103,9.108000000000034,136.62000000000052
United Kingdom,3,245.52000000000078,8.768571428571457,81.84000000000026
Portugal,2,185.13000000000022,6.3837931034482835,92.56500000000011
India,2,183.1500000000002,8.72142857142858,91.5750000000001
Other,15,1094.9400000000085,7.448571428571486,72.99600000000056


The analysis of sales data across different countries reveals interesting insights into customer distribution and sales performance. The United States (USA) stands out with the highest number of customers at 13, contributing significantly to the total sales of 1040.49. Despite having fewer customers, the Czech Republic shows a notably higher average order value and customer lifetime value compared to other countries. The "Other" category, which groups countries with only one customer, surprisingly contributes a substantial amount to total sales at 1094.94, albeit with a lower average order value. This analysis highlights the importance of not only focusing on large markets like the USA but also paying attention to smaller markets and their unique characteristics to optimise sales strategies effectively.

## Albums vs Individual Tracks

I will be analysing the purchasing behaviour of customers to determine the percentage of purchases that are individual tracks versus whole albums. This analysis is crucial to understand the potential impact of changing purchasing strategy to only buy the most popular tracks from each album.

To do this, I will first identify whether each invoice represents the purchase of a whole album or individual tracks by comparing the list of tracks from an invoice with the list of tracks from an album. I will use the EXCEPT operator to compare these lists and determine if an invoice contains all tracks from a specific album. Additionally, I will consider edge cases such as albums with few tracks or customers manually selecting tracks from multiple albums.

By using SQL queries and applying logical comparisons, I will create a comprehensive report that categorises each purchase as either an album purchase or a collection of individual tracks. This analysis will provide valuable insights to make informed decisions regarding purchasing strategy and revenue optimisation efforts.

In [10]:
%%sql
WITH 
invoices_first_track AS (
    -- Selecting the first track from each invoice
    SELECT invoice_id, MIN(track_id) AS first_track_id
      FROM invoice_line
  GROUP BY invoice_id
),
invoices_albums AS
(
    -- Categorising invoices as album purchases or not
    SELECT ift.*,
           CASE
           WHEN
                (
                  -- Checking if all tracks in the invoice belong to the same album
                  SELECT il.track_id 
                    FROM invoice_line AS il 
                   WHERE il.invoice_id = ift.invoice_id
                    
                  EXCEPT
                    
                  SELECT track_id 
                    FROM track 
                   WHERE album_id =
                      (
                        SELECT album_id 
                          FROM track 
                         WHERE track_id = ift.first_track_id
                      )
                ) IS NULL
    
                AND 
    
                (
                  -- Checking if all tracks in the album are in the invoice
                  SELECT track_id 
                    FROM track 
                   WHERE album_id =
                    (
                        SELECT album_id 
                          FROM track 
                         WHERE track_id = ift.first_track_id
                    )
                    
                 EXCEPT
                    
                 SELECT il.track_id 
                   FROM invoice_line AS il 
                  WHERE il.invoice_id = ift.invoice_id
               ) IS NULL THEN 'YES'
           ELSE 'NO'
           END AS album_purchase
    FROM invoices_first_track AS ift
)
  -- Calculating summary statistics
  SELECT
          album_purchase,
          COUNT(invoice_id) AS number_of_invoices,
          ROUND(CAST(COUNT(invoice_id) AS FLOAT) / 
                        (SELECT COUNT(*) FROM invoice),2) * 100 AS percent_of_invoices
    FROM invoices_albums 
GROUP BY album_purchase;

Done.


album_purchase,number_of_invoices,percent_of_invoices
NO,500,81.0
YES,114,19.0



The analysis indicates that a substantial portion of invoices (81.0%) are for non-album purchases, suggesting that customers prefer buying individual tracks over entire albums. Based on this data, I recommend that the Chinook store should consider adapting its purchasing strategy to concentrate more on acquiring individual tracks or popular tracks from albums rather than procuring complete albums from record companies. This adjustment in strategy has the potential to better align with customer preferences and enhance revenue generation. Regular monitoring and analysis of purchase patterns can aid in refining this strategy over time.

## Conclusion

In conclusion, this SQL portfolio project has successfully addressed various business questions using the Chinook sample database. The analysis included selecting new albums for purchase based on genre popularity in the USA, evaluating sales performance among sales support agents, analysing sales data by country, and examining purchasing behaviour between albums and individual tracks.

The findings from the analysis provide actionable insights for decision-making within the Chinook store. Recommendations include prioritising album purchases from Red Tone (Punk), Slim Jim Bites (Blues), and Meteor and Girls (Pop) based on genre popularity in the USA. The analysis of sales by country highlighted the importance of both large and small markets, emphasising the need for targeted marketing strategies. Additionally, the investigation into album versus individual track purchases suggests a shift towards acquiring popular tracks rather than complete albums to align with customer preferences and improve revenue generation.

Continued monitoring and analysis of customer behaviour and sales trends will be beneficial for optimising strategies and ensuring long-term business success in the digital media retail industry. Overall, leveraging SQL for data analysis has proven invaluable in gaining actionable insights and driving informed decision-making processes.